In [1]:
import numpy as np
def nms(boxes, scores, nms_thr, method):
    """Single class NMS implemented in Numpy."""
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]

    areas = (x2 - x1 + 1) * (y2 - y1 + 1)
    order = scores.argsort()[::-1]
    print(order)
    keep = []
    while order.size > 0:
        print(f'order size:{order.size}')
        i = order[0]
        keep.append(i)
        # 计算当前概率最大矩形框与 其他矩形框的相交框的坐标
        xx1 = np.maximum(x1[i], x1[order[1:]])
        yy1 = np.maximum(y1[i], y1[order[1:]])
        xx2 = np.minimum(x2[i], x2[order[1:]])
        yy2 = np.minimum(y2[i], y2[order[1:]])

        w = np.maximum(0.0, xx2 - xx1 + 1)
        h = np.maximum(0.0, yy2 - yy1 + 1)
        inter = w * h
        #数组，其他矩形框的iou与当前top框
        ovr = inter / (areas[i] + areas[order[1:]] - inter)

        #这一部分 hard 过滤了
        #np.where ,大于的抽离箱子
        print(f"inter : {inter}")
        print(f"ovr   : {ovr}")
        invalid = np.where(ovr > nms_thr)[0]


        if method == 2:
            sigma = 0.5
            weight = np.exp(-(ovr**2) / sigma)
            #重新打分
            scores[invalid] = [ x*weight for x in scores[invalid] ]
            print('method 2')
        else:
            inds = np.where(ovr <= nms_thr)[0]
            print(f'invalid inds:{invalid+1}')
            print(f'valid inds:{inds+1}')
            #order 索引
            order = order[inds + 1]

    return keep

In [3]:
import numpy as np
a = np.zeros(10)
id = [1,2,4] #list
c = np.array([100,100,100])
a[id] = c


In [39]:
import os
LOCAL_RANK = int(os.getenv('LOCAL_RANK', -1))  # https://pytorch.org/docs/stable/elastic/run.html
RANK = int(os.getenv('RANK', -1))
WORLD_SIZE = int(os.getenv('WORLD_SIZE', 1))
print(LOCAL_RANK)
print(RANK)
print(WORLD_SIZE)

-1
-1
1


# 2 IOu讨论及尝试 pytorch

In [4]:
ious  = np.arange(0.3,0.9,0.1)

sigma = 0.5
gaus_w = np.exp(-(ious**2) / sigma)
gd = {}
for k,v in zip(ious,gaus_w):
    gd[k] = v
liner_w = 1-ious
print(f'gaus_w : {gaus_w}')
print(f"liner_w : {liner_w}")
# 接触越少惩罚越小 当被包含时iou较小

gaus_w : [0.83527021 0.72614904 0.60653066 0.48675226 0.3753111  0.2780373
 0.1978987 ]
liner_w : [0.7 0.6 0.5 0.4 0.3 0.2 0.1]


In [24]:
ious_t = torch.from_numpy(ious)
a = ious_t.sum()
c = torch.randn(100,173400)
ones_matrix = torch.ones((1000,173400))
print(a)
print(c.sum(0).size())
print(c.sum(1).size())
anchor_matching_gt = ones_matrix.sum(1).int()#每一行 求和
print(anchor_matching_gt)

tensor(4.2000, dtype=torch.float64)
torch.Size([173400])
torch.Size([100])
tensor([173400, 173400, 173400, 173400, 173400, 173400, 173400, 173400, 173400,
        173400, 173400, 173400, 173400, 173400, 173400, 173400, 173400, 173400,
        173400, 173400, 173400, 173400, 173400, 173400, 173400, 173400, 173400,
        173400, 173400, 173400, 173400, 173400, 173400, 173400, 173400, 173400,
        173400, 173400, 173400, 173400, 173400, 173400, 173400, 173400, 173400,
        173400, 173400, 173400, 173400, 173400, 173400, 173400, 173400, 173400,
        173400, 173400, 173400, 173400, 173400, 173400, 173400, 173400, 173400,
        173400, 173400, 173400, 173400, 173400, 173400, 173400, 173400, 173400,
        173400, 173400, 173400, 173400, 173400, 173400, 173400, 173400, 173400,
        173400, 173400, 173400, 173400, 173400, 173400, 173400, 173400, 173400,
        173400, 173400, 173400, 173400, 173400, 173400, 173400, 173400, 173400,
        173400, 173400, 173400, 173400, 17340

In [17]:
print((480*272*85)/64.0)
print((160*160*85)/64.0)

173400.0
34000.0


In [3]:
box1 = [0,100,100,0]
box2 = [1,40,50,0]
iouv = torch.linspace(0.5, 0.95, 10)  # iou vector for mAP@0.5:0.95
niou = iouv.numel()

In [1]:
#torch实现各项 iou函数
import torch
import math
from loguru import logger
def bbox_iou_torch(box1:torch.Tensor, box2:torch.Tensor, x1y1x2y2=True, GIoU=False, DIoU=False, CIoU=False, eps=1e-9):
    # Returns the IoU of box1 to box2. box1 is 4, box2 is nx4
    box2 = box2.T
    logger.info(box2.shape)
    # Get the coordinates of bounding boxes
    if x1y1x2y2:  # x1, y1, x2, y2 = box1
        b1_x1, b1_y1, b1_x2, b1_y2 = box1[0], box1[1], box1[2], box1[3]
        b2_x1, b2_y1, b2_x2, b2_y2 = box2[0], box2[1], box2[2], box2[3]
    else:  # transform from xywh to xyxy
        b1_x1, b1_x2 = box1[0] - box1[2] / 2, box1[0] + box1[2] / 2
        b1_y1, b1_y2 = box1[1] - box1[3] / 2, box1[1] + box1[3] / 2
        b2_x1, b2_x2 = box2[0] - box2[2] / 2, box2[0] + box2[2] / 2
        b2_y1, b2_y2 = box2[1] - box2[3] / 2, box2[1] + box2[3] / 2
 
    # Intersection area
    inter = (torch.min(b1_x2, b2_x2) - torch.max(b1_x1, b2_x1)).clamp(0) * \
            (torch.min(b1_y2, b2_y2) - torch.max(b1_y1, b2_y1)).clamp(0)
 
    # Union Area
    w1, h1 = b1_x2 - b1_x1, b1_y2 - b1_y1 + eps
    w2, h2 = b2_x2 - b2_x1, b2_y2 - b2_y1 + eps
    union = w1 * h1 + w2 * h2 - inter + eps
    logger.info(f"inter {inter} / union {union} = iou {inter/union}")
    iou = inter / union
    if GIoU or DIoU or CIoU:
        cw = torch.max(b1_x2, b2_x2) - torch.min(b1_x1, b2_x1)  # convex (smallest enclosing box) width
        ch = torch.max(b1_y2, b2_y2) - torch.min(b1_y1, b2_y1)  # convex height
        if CIoU or DIoU:  # Distance or Complete IoU https://arxiv.org/abs/1911.08287v1
            c2 = cw ** 2 + ch ** 2 + eps  # convex diagonal squared
            rho2 = ((b2_x1 + b2_x2 - b1_x1 - b1_x2) ** 2 +
                    (b2_y1 + b2_y2 - b1_y1 - b1_y2) ** 2) / 4  # center distance squared
            if DIoU:
                return iou - rho2 / c2  # DIoU
            elif CIoU:  # https://github.com/Zzh-tju/DIoU-SSD-pytorch/blob/master/utils/box/box_utils.py#L47
                v = (4 / math.pi ** 2) * torch.pow(torch.atan(w2 / h2) - torch.atan(w1 / h1), 2)
                with torch.no_grad():
                    alpha = v / ((1 + eps) - iou + v)
                return iou - (rho2 / c2 + v * alpha)  # 3 CIoU
        else:  # GIoU https://arxiv.org/pdf/1902.09630.pdf
            c_area = cw * ch + eps  # convex area
            return iou - (c_area - union) / c_area  # 2 GIoU
    else:
        return iou  #  1 IoU

# numpy版本实现iou
# torch lb rt
import torch
box1 = [0,0,100,100]
box2 = [[0,0,50,40] ,[20,10,90,40]]
box1 = torch.Tensor(box1)
box2 = torch.Tensor(box2)
iou = bbox_iou_torch(box1 ,box2)

2023-01-06 12:57:32.858 | INFO     | __main__:bbox_iou_torch:8 - torch.Size([4, 2])
2023-01-06 12:57:32.895 | INFO     | __main__:bbox_iou_torch:27 - inter tensor([2000., 2100.]) / union tensor([10000., 10000.]) = iou tensor([0.2000, 0.2100])


In [4]:
box2.shape[1]

4

## pytorch 数学函数

In [16]:
import numpy as np
pred = torch.tensor([[-0.5816, -0.3873, -1.0215, -1.0145,  0.4053],
        [ 0.7265,  1.4164,  1.3443,  1.2035,  np.nan],
        [-0.4451,  0.1673,  1.2590, -2.0757,  1.7255],
        [ 0.2021,  0.3041,  0.1383,  0.3849, -1.6311]])
# 0 按列， 1按行
# pred = pred.add(1)
pred = torch.where(torch.isnan(pred), torch.full_like(pred, 0), pred)
vv,vi = pred.topk(k=3,dim=1,largest=True,sorted=True)
dyn_ks = torch.clamp(vv.sum(1).int(), min=1)
dyn_ks = dyn_ks.tolist()

assert not torch.isnan(pred).any(), 'pred exist nan'

In [8]:
tmp = [[1.0,3.0,2,4],[2,2,3,2]]
a = torch.FloatTensor(tmp)
a1 =torch.HalfTensor(tmp)
a2 = torch.DoubleTensor(tmp)
a3 = torch.IntTensor(tmp)
a4 = torch.LongTensor(tmp)
# print(a4.dtype)
loss = torch.log(a4+ 1e-8)
print(loss)

tensor([[0.0000, 1.0986, 0.6931, 1.3863],
        [0.6931, 0.6931, 1.0986, 0.6931]])


<ipython-input-8-704fa63e612d>:5: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  a3 = torch.IntTensor(tmp)
<ipython-input-8-704fa63e612d>:6: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  a4 = torch.LongTensor(tmp)


In [1]:
# 官方实现多对多
import torch
box1 = [[0,0,100,100]]
box2 = [[0,0,50,40] ,[20,10,90,40]]
box1 = torch.Tensor(box1)
box2 = torch.Tensor(box2)
# iou = bbox_iou_torch(box1 ,box2)

from torchvision.ops import box_iou, complete_box_iou,distance_box_iou,generalized_box_iou
iou = box_iou(boxes1=box1, boxes2=box2)
ciou = complete_box_iou(box1,box2)
diou = distance_box_iou(box1, box2)
giou = generalized_box_iou(box1, box2)

In [47]:
dets = [[0, 0, 100, 101, 0.9], [5, 6, 90, 110, 0.7], [17, 19, 80, 120, 0.8], [10, 8, 115, 105, 0.5],[114, 8, 115, 105, 0.7]]
dets = np.array(dets)
result = nms(dets[:,0:4],dets[:,-1], 0.6,method=1)
print(f'result:{result}')
print('@'*100)
# result = nms(dets[:,0:4],dets[:,-1], 0.6, method=2)
# print(f'm2 result:{result}')
print('@'*100)

[0 2 4 1 3]
order size:5
inter : [5312.    0. 8256. 8554.]
ovr   : [0.46119118 0.         0.74539545 0.70484509]
invalid inds:[3 4]
valid inds:[1 2]
order size:2
inter : [0.]
ovr   : [0.]
invalid inds:[]
valid inds:[1]
order size:1
inter : []
ovr   : []
invalid inds:[]
valid inds:[]
result:[0, 2, 4]
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
soft-nms result:[0, 1, 2, 3]


In [43]:
def soft_nms(dets, sigma=0.5, nms_thr=0.5, method=2,score_thr=0.1):
    box_len = len(dets)   # box的个数
    for i in range(box_len):
        tmpx1, tmpy1, tmpx2, tmpy2, ts = dets[i, 0], dets[i, 1], dets[i, 2], dets[i, 3], dets[i, 4]
        max_pos = i
        max_scores = ts
        # get 最大置信度的box
        pos = i+1
        while pos < box_len:
            if max_scores < dets[pos, 4]:
                max_scores = dets[pos, 4]
                max_pos = pos
            pos += 1
        # add max box as a detection
        dets[i, :] = dets[max_pos, :]
        # swap ith box with position of max box 交换，把 maxbox 替换到首位
        dets[max_pos, 0] = tmpx1
        dets[max_pos, 1] = tmpy1
        dets[max_pos, 2] = tmpx2
        dets[max_pos, 3] = tmpy2
        dets[max_pos, 4] = ts
        # 将置信度最高的 box 赋给临时变量
        tmpx1, tmpy1, tmpx2, tmpy2, ts = dets[i, 0], dets[i, 1], dets[i, 2], dets[i, 3], dets[i, 4]
        pos = i+1
        # NMS iterations, note that box_len changes if detection boxes fall below threshold
        while pos < box_len:
            x1, y1, x2, y2 = dets[pos, 0], dets[pos, 1], dets[pos, 2], dets[pos, 3]
            area = (x2 - x1 + 1)*(y2 - y1 + 1)
            iw = (min(tmpx2, x2) - max(tmpx1, x1) + 1)
            ih = (min(tmpy2, y2) - max(tmpy1, y1) + 1)
            if iw > 0 and ih > 0:
                overlaps = iw * ih
                ious = overlaps / ((tmpx2 - tmpx1 + 1) * (tmpy2 - tmpy1 + 1) + area - overlaps)
                #两种权值方法
                if method == 1:    # 1.线性
                    if ious > nms_thr:
                        weight = 1 - ious
                    else:
                        weight = 1
                elif method == 2:  # 2.gaussian
                    weight = np.exp(-(ious**2) / sigma)
                else:              # 3.hard NMS
                    if ious > nms_thr:
                        weight = 0
                    else:
                        weight = 1

                # 赋予该box新的置信度
                dets[pos, 4] = weight * dets[pos, 4]

                # 如果box得分低于阈值thresh，则通过与最后一个框交换来丢弃该框
                if dets[pos, 4] < score_thr:
                    dets[pos, 0] = dets[box_len-1, 0]
                    dets[pos, 1] = dets[box_len-1, 1]
                    dets[pos, 2] = dets[box_len-1, 2]
                    dets[pos, 3] = dets[box_len-1, 3]
                    dets[pos, 4] = dets[box_len-1, 4]

                    box_len = box_len-1
                    pos = pos-1
            pos += 1

    keep = [i for i in range(box_len)]
    return keep

In [49]:
a=[[2,1,3],[23,23,1]]
b=[[1],[2]]
for i in range(2):
    a[i].append(b[i][0])

In [3]:
import numpy as np 
 
x = np.arange(9.).reshape(1,3, 3)  
print ('我们的数组是：')
print (x)
print ( '大于 3 的元素的索引：')
y = np.where(x >  3) 
y2 = np.where(x >  3)[0] 
#返回一个元组
y1 = np.concatenate(y)
print (y1.size)
print ('使用这些索引来获取满足条件的元素：')
print (x[y])
#三目表达式
y = np.arange(9.) 
x1=np.where(x>3, x, x+3)[0]
y=np.where(y>3, y, y*2)

我们的数组是：
[[[0. 1. 2.]
  [3. 4. 5.]
  [6. 7. 8.]]]
大于 3 的元素的索引：
15
使用这些索引来获取满足条件的元素：
[4. 5. 6. 7. 8.]


In [19]:
ovr = np.arange(9)
# weight = np.(ovr)
y=np.where(y>6, y, y*ovr)
y.size

9

In [25]:
a = np.arange(20)
b = np.arange(20,40)
id = [1,2,3,4,5]
weight = np.linspace(1,10,20)
a[id] = a[id] * weight[id]
a+1


array([ 1,  2,  4,  8, 12, 17,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20])

In [4]:
x = np.arange(60).reshape(12,-1)
x1 = np.arange(20).reshape(4,-1)
np.argsort(x,axis=0)
# np.argsort(x,axis=2)
np.argsort(x,axis=1)
x2 = np.concatenate((x,x1))
##
a = [1,2,3]
an =np.array(a)
print(x2[a,:4])

[[ 5  6  7  8]
 [10 11 12 13]
 [15 16 17 18]]


In [16]:
x1[:,3] =[1,2,10,10000]
x11 = x1[np.argsort(x1[:,3])[::-1]]
order = x[1:,...]
a=[]
keep1 = np.empty((0,5), int)
a.append(x1)
a.append(x2)
# an = np.array(a)
#拼凑array
keep1 = np.empty((0,5), int)
keep1 = np.concatenate((keep1,x1))
keep1 = np.concatenate((keep1,x))

In [18]:
class_a = True
print(type(class_a))
if class_a:
    print(1)
else:
    print(2)

<class 'bool'>
1
